In [16]:
import pandas as pd
import os
import re
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

# PATHS 

HERE = Path.cwd()
REPO_ROOT = next(p for p in [HERE] + list(HERE.parents) if (p / "Data").exists())

INPUT_DIR = REPO_ROOT / "Data" / "NNDSS"   # read .xlsx from here
OUTPUT_DIR = REPO_ROOT / "Data"            # save merged file here

# Find Excel files
excel_files = sorted([f for f in os.listdir(INPUT_DIR) if f.endswith('.xlsx')])

print(f"Processing {len(excel_files)} files from: {INPUT_DIR}\n")

# HELPER FUNCTIONS 

def get_fortnight_from_file(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name=0, header=None, nrows=10)

        for i in range(min(10, len(df))):
            for j in range(min(10, len(df.columns))):
                cell = str(df.iloc[i, j])
                fn_match = re.search(r'FN(\d+)[/\s]*(\d{4})', cell, re.IGNORECASE)
                if fn_match:
                    return int(fn_match.group(1)), int(fn_match.group(2))

        filename = os.path.basename(file_path)
        fn_match = re.search(r'fn(\d+)', filename, re.IGNORECASE)
        year_match = re.search(r'(202[3-9])', filename)

        if fn_match and year_match:
            fn_num = int(fn_match.group(1))
            year = int(year_match.group(1))
            return fn_num, year

        months = {'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6,
                  'july': 7, 'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12}

        for month_name, month_num in months.items():
            if month_name in filename.lower():
                day_match = re.search(r'(\d{1,2})[_\s-]' + month_name, filename, re.IGNORECASE)
                if day_match and year_match:
                    day = int(day_match.group(1))
                    fn_num = (month_num - 1) * 2 + (1 if day <= 15 else 2)
                    return fn_num, int(year_match.group(1))

        return None, None
    except:
        return None, None


def read_nndss_file(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name=0, header=None)

        header_row = None
        for idx, row in df.iterrows():
            row_str = ' '.join([str(x) for x in row.values if pd.notna(x)])
            if 'Disease name' in row_str:
                header_row = idx
                break

        if header_row is None:
            return None

        df.columns = df.iloc[header_row]
        data_start = header_row + 1

        while data_start < len(df) and data_start < header_row + 5:
            if pd.isna(df.iloc[data_start, 0]) or 'datetime' in str(df.iloc[data_start, 0]).lower():
                data_start += 1
            else:
                break

        data_df = df.iloc[data_start:].copy()

        disease_col = None
        for col in data_df.columns:
            if pd.notna(col) and 'Disease name' in str(col):
                disease_col = col
                break

        total_col = None
        for col in data_df.columns:
            if pd.notna(col) and 'This reporting period' in str(col):
                total_col = col
                break

        if disease_col is None or total_col is None:
            return None

        result_df = data_df[[disease_col, total_col]].copy()
        result_df.columns = ['Disease', 'Total_Cases']

        result_df = result_df[result_df['Disease'].notna()]
        result_df['Disease'] = result_df['Disease'].astype(str).str.strip()
        result_df = result_df[~result_df['Disease'].str.lower().str.endswith('diseases')]
        result_df = result_df[result_df['Disease'] != 'nan']

        result_df['Total_Cases'] = pd.to_numeric(result_df['Total_Cases'], errors='coerce')
        result_df = result_df[result_df['Total_Cases'].notna()]

        return result_df
    except:
        return None


# PROCESS ALL FILES

all_data = {}
processed_fortnights = {}

for filename in excel_files:
    file_path = str(INPUT_DIR / filename)

    fn_num, year = get_fortnight_from_file(file_path)

    if fn_num and year:
        column_name = f"FN{fn_num:02d}_{year}"

        if column_name in processed_fortnights:
            continue

        df = read_nndss_file(file_path)

        if df is not None and len(df) > 0:
            processed_fortnights[column_name] = filename

            for _, row in df.iterrows():
                disease = row['Disease']
                total = row['Total_Cases']

                if disease not in all_data:
                    all_data[disease] = {}

                all_data[disease][column_name] = total

print(f"Successfully processed {len(processed_fortnights)} fortnights\n")


# CREATE MERGED DATAFRAME

merged_df = pd.DataFrame(all_data).T

def sort_key(col):
    match = re.search(r'FN(\d+)_(\d{4})', col)
    if match:
        return (int(match.group(2)), int(match.group(1)))
    return (9999, 99)

sorted_columns = sorted(merged_df.columns, key=sort_key)
merged_df = merged_df[sorted_columns]

merged_df.reset_index(inplace=True)
merged_df.rename(columns={'index': 'Disease_Name'}, inplace=True)
merged_df = merged_df.fillna(0)

print(f"Final dataset: {merged_df.shape[0]} diseases × {merged_df.shape[1]-1} fortnights")


# SAVE ONLY THE MERGED FILE

output_file = str(OUTPUT_DIR / "AUS_merged_nndss_all_diseases.xlsx")
merged_df.to_excel(output_file, index=False, engine='openpyxl')

print(f"\n{'='*70}")
print(f"✓ FILE SAVED: {output_file}")
print(f"  Size: {os.path.getsize(output_file):,} bytes")
print(f"{'='*70}")

print("\n✓ COMPLETE! Output saved under Data/: merged_nndss_all_diseases.xlsx")

Processing 49 files from: /workspaces/group-project-inferra/Data/NNDSS

Successfully processed 49 fortnights

Final dataset: 76 diseases × 49 fortnights

✓ FILE SAVED: /workspaces/group-project-inferra/Data/AUS_merged_nndss_all_diseases.xlsx
  Size: 19,199 bytes

✓ COMPLETE! Output saved under Data/: merged_nndss_all_diseases.xlsx


In [17]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

# PATHS

HERE = Path.cwd()
REPO_ROOT = next(p for p in [HERE] + list(HERE.parents) if (p / "Data").exists())

MERGED_FILE = REPO_ROOT / "Data" / "AUS_merged_nndss_all_diseases.xlsx"

# Load merged dataset

df = pd.read_excel(MERGED_FILE, engine="openpyxl")

print("\n=== BASIC INFO ===")
print("Shape:", df.shape)
print("Columns:", df.columns.tolist()[:10], "...")
print(df.head())


# Basic exploration (like COVID)

print("\n=== MISSINGNESS (top 10 columns) ===")
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
print(missing_pct.head(10))

print("\n=== DESCRIPTIVE STATS (first few FN columns) ===")
num_cols = [c for c in df.columns if c.startswith("FN")]
print(df[num_cols[:5]].describe())



=== BASIC INFO ===
Shape: (76, 50)
Columns: ['Disease_Name', 'FN01_2024', 'FN02_2024', 'FN03_2024', 'FN04_2024', 'FN05_2024', 'FN06_2024', 'FN07_2024', 'FN08_2024', 'FN09_2024'] ...
                   Disease_Name  FN01_2024  FN02_2024  FN03_2024  FN04_2024  \
0  Hepatitis B (newly acquired)          4          2          0          5   
1     Hepatitis B (unspecified)        303        188        206        210   
2  Hepatitis C (newly acquired)         58         22         24         24   
3     Hepatitis C (unspecified)        466        279        235        272   
4                   Hepatitis D          4          2          5          6   

   FN05_2024  FN06_2024  FN07_2024  FN08_2024  FN09_2024  ...  FN15_2025  \
0          2          4          2          4          3  ...          2   
1        223        222        242        251        259  ...        196   
2         34         29         26         13         14  ...         22   
3        279        291        264    

In [18]:
import pandas as pd
import re
from pathlib import Path


# PATHS
HERE = Path.cwd()
REPO_ROOT = next(p for p in [HERE] + list(HERE.parents) if (p / "Data").exists())

MERGED_FILE = REPO_ROOT / "Data" / "AUS_merged_nndss_all_diseases.xlsx"
OUT_DIR = REPO_ROOT / "Data"

df = pd.read_excel(MERGED_FILE, engine="openpyxl")

fn_cols = [c for c in df.columns if re.match(r"FN\d+_\d{4}", str(c))]
id_col = "Disease_Name"

def extract_and_save_xlsx(disease_label: str, patterns: list[str]):
    mask = df[id_col].astype(str).str.contains("|".join(patterns), case=False, na=False, regex=True)
    sub = df[mask].copy()

    if sub.empty:
        print(f"❌ {disease_label}: not found. Try updating patterns.")
        return

    out_file = OUT_DIR / f"AUS_{disease_label.lower().replace(' ', '_')}.xlsx"
    sub.to_excel(out_file, index=False, engine="openpyxl")

    print(f"✅ Saved {disease_label} to: {out_file}")
    print(f"   Rows: {len(sub)}, FN columns: {len(fn_cols)}")

# Save both files
extract_and_save_xlsx("Measles", [r"\bmeasles\b"])
extract_and_save_xlsx("RSV", [r"\brsv\b", r"respiratory\s+syncytial\s+virus"])

✅ Saved Measles to: /workspaces/group-project-inferra/Data/AUS_measles.xlsx
   Rows: 1, FN columns: 49
✅ Saved RSV to: /workspaces/group-project-inferra/Data/AUS_rsv.xlsx
   Rows: 1, FN columns: 49
